In [16]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D

model=Sequential()

model.add(Conv2D(256,(3,3),input_shape=[50,50,1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(64,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(4,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
model.load_weights('../res/Models/modelAgeGender.h5')

In [18]:
import cv2, pickle
import dlib
import numpy as np

In [19]:
cam = cv2.VideoCapture('../res/test_data/videos/vid.mp4')#rtsp://192.168.1.6/live/ch00_1 'https://192.168.1.5:8080/video'

dict_file = open("../res/Data/cat_dict.pkl", "rb")
category_dict = pickle.load(dict_file)

face_detector = dlib.get_frontal_face_detector()
margin = 20

In [20]:
while(True):
    ret, feed = cam.read()
    feed = cv2.resize(feed, (852, 480))
    
    faces = face_detector(feed)
    
    try:
        for face in faces:

            x1 = (face.left()) - margin
            y1 = (face.top()) - margin
            x2 = (face.right()) + margin
            y2 = (face.bottom()) + margin

            test_img = feed[y1:y2, x1:x2]

            test_img = cv2.resize(test_img, (50,50))
            test_img = cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)
            test_img = test_img/255
            test_img = test_img.reshape(1, 50, 50, 1)

            results = model.predict(test_img)
            label = np.argmax(results, axis=1)[0]
            acc = int(np.max(results, axis=1)[0]*100)

            print(results,label,category_dict[label],acc)

            cv2.rectangle(feed, (x1, y1), (x2, y2),(0, 255, 0), 2)
            cv2.rectangle(feed, (x1-1, y1-20), (x1+99, y1), (0, 255, 0), -1)
            #cv2.putText(feed, "Face", (x1, y1-1), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0 ,0, 255), 2)

            cv2.putText(feed, f"{category_dict[label]}({str(acc)}%)", (x1, y1-1), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)

    except:
        pass
    
    cv2.imshow('LIVE_FEED',feed)
    k=cv2.waitKey(1)
    if(k==27):
        break
cv2.destroyAllWindows()

[[1.3922945e-12 6.0450914e-04 1.5128537e-11 9.9939549e-01]] 3 40+ 99
[[1.3922945e-12 6.0450914e-04 1.5128537e-11 9.9939549e-01]] 3 40+ 99
[[9.4423623e-13 4.1346758e-04 2.5388464e-11 9.9958652e-01]] 3 40+ 99
[[1.7401694e-08 2.9547398e-03 1.3536882e-07 9.9704510e-01]] 3 40+ 99
[[6.7273448e-16 3.6638390e-04 1.3982637e-23 9.9963355e-01]] 3 40+ 99
[[5.2252183e-21 6.4329684e-09 5.1102833e-25 1.0000000e+00]] 3 40+ 100
[[1.8131512e-22 3.7315248e-10 2.2999509e-26 1.0000000e+00]] 3 40+ 100
[[1.16748425e-20 4.93940755e-09 2.43042917e-25 1.00000000e+00]] 3 40+ 100
[[9.403293e-18 8.248491e-06 1.527308e-23 9.999918e-01]] 3 40+ 99
[[4.8466024e-16 2.2051037e-05 2.7160576e-21 9.9997795e-01]] 3 40+ 99
[[4.0293310e-16 7.3196156e-06 5.3408596e-19 9.9999273e-01]] 3 40+ 99
[[1.0441762e-12 3.1847834e-05 1.3037534e-20 9.9996817e-01]] 3 40+ 99
[[9.9510255e-13 5.4976390e-05 6.8287540e-20 9.9994504e-01]] 3 40+ 99
[[1.2918379e-12 6.5765285e-05 6.5657533e-19 9.9993420e-01]] 3 40+ 99
[[2.3238867e-11 1.8088971e-04 8

[[9.3885255e-12 9.5883340e-01 4.8758558e-18 4.1166630e-02]] 1 27-40 95
[[5.2340425e-07 7.3022908e-01 7.8326248e-06 2.6976255e-01]] 1 27-40 73
[[4.2331840e-07 8.1668782e-01 1.0878281e-05 1.8330091e-01]] 1 27-40 81
[[9.9999690e-01 5.5038993e-07 2.3290388e-06 2.0527925e-07]] 0 13-26 99
[[9.9999714e-01 1.3389072e-06 1.4917982e-06 4.7894726e-08]] 0 13-26 99
[[9.9998415e-01 1.5713715e-05 8.6192372e-08 3.8321605e-08]] 0 13-26 99
[[1.0680194e-02 9.8839825e-01 2.7659409e-12 9.2156214e-04]] 1 27-40 98
[[4.6741934e-03 9.9496740e-01 1.2557713e-12 3.5845852e-04]] 1 27-40 99
[[6.4884627e-04 9.9666411e-01 2.2338414e-11 2.6870193e-03]] 1 27-40 99
[[8.8772446e-01 1.1227465e-01 4.1976302e-18 9.0038486e-07]] 0 13-26 88
[[8.2190126e-01 1.7809799e-01 6.1584978e-18 6.7981637e-07]] 0 13-26 82
[[5.2667521e-05 3.0320272e-01 1.2425397e-10 6.9674462e-01]] 3 40+ 69
[[6.7200077e-01 2.4528424e-01 3.7195337e-11 8.2714975e-02]] 0 13-26 67
[[6.5615875e-01 2.7301431e-01 8.4784645e-11 7.0826970e-02]] 0 13-26 65
[[9.9968